## 🔧 步骤1: 环境配置

首先配置必要的环境和API密钥

In [5]:
import os
import sys
from pathlib import Path

# 添加项目路径
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

# 配置API密钥（请替换为你的密钥）
# 方式1: 直接设置（不推荐，仅用于测试）
# os.environ['GEMINI_API_KEY'] = 'your_api_key_here'

# 方式2: 从.env文件读取（推荐）
from dotenv import load_dotenv
load_dotenv()

# 验证
api_key = os.getenv('GEMINI_API_KEY')
if api_key:
    print("✅ API密钥已配置")
    print(f"   密钥前缀: {api_key[:10]}...")
else:
    print("⚠️  未找到API密钥，将使用模拟模式")
    print("   提示: 在.env文件中设置 GEMINI_API_KEY=your_key")

✅ API密钥已配置
   密钥前缀: AIzaSyCglf...


## 📁 步骤2: 选择目标项目

可以选择:
1. GitHub公开项目（自动克隆）
2. 本地项目路径

In [6]:
from src.context_analyzer import GitHubIntegration

# 方式1: 使用GitHub项目（推荐）
# 示例：本体匹配项目
project_source = "https://github.com/qzc438-research/ontology-llm"

# 方式2: 使用本地项目
# project_source = "/Users/yourname/projects/your-project"

# 克隆或使用项目
print("🔄 准备项目...")
project_path = GitHubIntegration.clone_or_use_repo(project_source)

print(f"\n📂 项目路径: {project_path}")
print(f"   项目名称: {project_path.name}")

# 快速浏览项目结构
python_files = list(project_path.rglob('*.py'))
python_files = [f for f in python_files if '__pycache__' not in str(f) and '.venv' not in str(f)]
print(f"   Python文件: {len(python_files)} 个")

🔄 准备项目...
✅ 项目已存在: /Users/xianhaoliu/github_repos/ontology-llm

📂 项目路径: /Users/xianhaoliu/github_repos/ontology-llm
   项目名称: ontology-llm
   Python文件: 13 个


## 🧠 步骤3: 初始化生成器

创建简单生成器实例，支持上下文增强

In [7]:
from src.simple_generator import SimpleGenerator

# 创建生成器
generator = SimpleGenerator(
    project_path=str(project_path),
    model="gemini-2.5-flash",
    temperature=0.3  # 较低温度保证格式稳定性
)

print("✅ 生成器初始化成功")
print(f"   LLM可用: {generator.llm_available}")
print(f"   上下文增强: {generator.context_enabled}")
print(f"   项目: {generator.project_path.name}")

✅ 生成器初始化成功
   LLM可用: True
   上下文增强: True
   项目: ontology-llm


## 📝 步骤4a: 生成单个问答对（测试）

先生成一个问答对测试效果

In [8]:
import random

# 随机选择一个文件
files = generator.discover_python_files()
test_file = random.choice(files)
rel_path = test_file.relative_to(project_path)

print(f"📄 测试文件: {rel_path}\n")

# 提取代码片段
code_snippet = generator.extract_code_snippet(test_file, length=600)
print("【代码片段预览】")
print(code_snippet[:300] + "...\n")

# 生成问答对
print("🔄 生成问答对...")
qa = generator.generate_qa_pair(
    code_snippet=code_snippet,
    file_path=str(rel_path),
    use_context=True  # 启用上下文增强
)

if qa:
    print("\n✅ 生成成功！\n")
    print("="*70)
    print(f"问题: {qa['question']}\n")
    print(f"答案: {qa['answer']}\n")
    print("推理步骤:")
    for i, step in enumerate(qa['reasoning_steps'], 1):
        print(f"  {i}. {step}")
    print("="*70)
else:
    print("❌ 生成失败，请检查API配置")

📄 测试文件: llm_om_few_shot.py

【代码片段预览】
ty.callbacks import get_openai_callback

import run_config as config
import om_ontology_to_csv
import util


# customer settings
o1_path = config.o1_path
o2_path = config.o2_path
align_path = config.align_path
context = config.context
o1_is_code = config.o1_is_code
o2_is_code = config.o2_is_code

o1...

🔄 生成问答对...

✅ 生成成功！

问题: 在 `ontology-llm` 项目中，特别是 `llm_om_few_shot.py` 模块内，定义了 `extract_yes_no` 函数。鉴于项目核心模块如 `generate_conference_benchmark` 和 `llm_matching` 的存在，请阐述 `extract_yes_no` 函数在整体 LLM-based 本体对齐和评估管道中的具体作用和重要性。该函数如何实现在 Few-shot 和 Zero-shot 场景下对齐质量的定量评估，以及它对提示工程和结果解释引入了哪些架构上的考虑？

答案: `extract_yes_no` 函数在 `ontology-llm` 项目中扮演着将 LLM 的自由文本响应转化为可量化、结构化二元决策的关键角色，这对于本体对齐的自动化评估至关重要。

1.  **`extract_yes_no` 的核心作用:**
    该函数的主要目的是从 LLM 的自然语言输出中可靠地提取出明确的“Yes”或“No”决策。它使用正则表达式 (`re.search`) 来识别文本中表示肯定或否定的词语。这种二元提取是 LLM 进行分类或决策任务的常见后处理步骤，确保了后续处理的一致性。

2.  **在 LLM-based 本体对齐工作流中的集成:**
    *   **本体实体对比较:** `llm_matching` 模块很可能负责将一对本体实体（如概念、属性或关系）作为输入，构建成一个自

## 🏗️ 步骤4b: 生成单个设计方案（测试）

In [9]:
# 定义需求
test_requirement = "为项目添加批量处理功能，支持并发处理大量数据"

print(f"📋 测试需求: {test_requirement}\n")

# 生成设计方案
print("🔄 生成设计方案...")
design = generator.generate_design_solution(
    requirement=test_requirement,
    use_context=True
)

if design:
    print("\n✅ 生成成功！\n")
    print("="*70)
    print(f"需求: {design['requirement']}\n")
    print(f"解决方案: {design['solution']}\n")
    print("实施步骤:")
    for i, step in enumerate(design['steps'], 1):
        print(f"  {i}. {step}")
    print("\n需要修改的文件:")
    for item in design['files_to_modify']:
        print(f"  - {item['file']}: {item['reason']}")
    print("="*70)
else:
    print("❌ 生成失败，请检查API配置")

📋 测试需求: 为项目添加批量处理功能，支持并发处理大量数据

🔄 生成设计方案...

✅ 生成成功！

需求: 为项目添加批量处理功能，支持并发处理大量数据

解决方案: 结合 `ontology-llm` 项目的核心模块，我们将通过引入 Python 的 `concurrent.futures` 模块来实现批量并发处理。具体而言，我们将把单个本体匹配任务（由 `llm_om_zero_shot` 或 `llm_matching` 模块执行）封装为可并行执行的单元。`run_series_conference` 模块将作为批处理的调度器，负责加载任务列表、创建线程池（或进程池，取决于任务性质），分发任务，并聚合结果。`util` 模块将提供辅助函数，如任务配置解析、进度管理和结果保存。

实施步骤:
  1. **定义批处理任务单元结构:**
    *   确定一个标准的数据结构（例如，一个字典或一个Pydantic模型）来表示一个独立的本体匹配任务。这个结构应该包含 `llm_om_zero_shot` 或 `llm_matching` 模块执行一次匹配所需的所有参数（例如，本体对ID、模型配置、输入文件路径、输出路径等）。
    *   **示例:** `{"task_id": "conference_pair_1_0", "ontology_a_path": "data/conf/conf_1.owl", "ontology_b_path": "data/conf/conf_0.owl", "llm_config": {"model_name": "gpt-3.5-turbo", "temperature": 0.7}}`
  2. **封装核心处理逻辑为独立函数:**
    *   在 `llm_om_zero_shot.py` (和/或 `llm_matching.py`) 中，将执行单个本体匹配的逻辑封装成一个独立的、无状态的函数。这个函数将接收步骤1中定义的任务单元结构作为参数，并返回该任务的处理结果（例如，匹配结果、评估指标、日志信息）。
    *   **示例:** `def process_single_matching_task(task_config: Dict) -> Dict: ...` 确保这个函数内部能处理所有必要的加载、

## 🚀 步骤5: 批量生成训练数据

一键生成完整训练数据集

In [10]:
# 配置生成参数
NUM_QA_PAIRS = 10      # 问答对数量
NUM_DESIGN = 5         # 设计方案数量
USE_CONTEXT = True     # 是否启用上下文增强

print("="*70)
print("🎯 批量生成配置")
print("="*70)
print(f"   问答对数量: {NUM_QA_PAIRS}")
print(f"   设计方案数量: {NUM_DESIGN}")
print(f"   上下文增强: {USE_CONTEXT}")
print(f"   预计耗时: {(NUM_QA_PAIRS + NUM_DESIGN) * 15 / 60:.1f} 分钟\n")

# 执行批量生成
dataset = generator.generate_batch(
    num_qa=NUM_QA_PAIRS,
    num_design=NUM_DESIGN,
    use_context=USE_CONTEXT
)

print("\n🎉 批量生成完成！")

🎯 批量生成配置
   问答对数量: 10
   设计方案数量: 5
   上下文增强: True
   预计耗时: 3.8 分钟

🚀 开始生成训练数据

📁 发现Python文件...
   找到 13 个文件

📝 生成 10 个问答对...
   [1/10] 处理文件: run_series_conference.py
       ✅ 成功
   [2/10] 处理文件: llm_om_few_shot.py
       ✅ 成功
   [3/10] 处理文件: llm_matching.py
       ✅ 成功
   [4/10] 处理文件: generate_anatomy_mse_benchmark.py
       ✅ 成功
   [5/10] 处理文件: llm_matching.py
       ✅ 成功
   [6/10] 处理文件: fix_multifarm_reference.py
       ✅ 成功
   [7/10] 处理文件: generate_anatomy_mse_benchmark.py
       ✅ 成功
   [8/10] 处理文件: fix_multifarm_reference.py
       ✅ 成功
   [9/10] 处理文件: generate_anatomy_mse_benchmark.py
       ✅ 成功
   [10/10] 处理文件: generate_anatomy_mse_benchmark.py
       ✅ 成功

🏗️  生成 5 个设计方案...
   [1/5] 需求: 扩展generate_conference_benchmark以支持弱网环境场景...
       ✅ 成功
   [2/5] 需求: 重构om_csv_to_database以支持国际化...
       ✅ 成功
   [3/5] 需求: 为fix_multifarm_reference添加错误处理功能...
       ✅ 成功
   [4/5] 需求: 集成gRPC到generate_conference_benchmark中...
       ✅ 成功
   [5/5] 需求: 为om_csv_to_database添加配置管理功能...
       ✅ 成功

📊

## 📊 步骤6: 数据质量分析

分析生成数据的质量指标

In [15]:
import json
from collections import Counter

# 统计问答对
print("="*70)
print("📊 问答对质量分析")
print("="*70)

if dataset['qa_pairs']:
    # 基础统计
    total_qa = len(dataset['qa_pairs'])
    avg_question_len = sum(len(qa['question']) for qa in dataset['qa_pairs']) / total_qa
    avg_answer_len = sum(len(qa['answer']) for qa in dataset['qa_pairs']) / total_qa
    avg_reasoning_steps = sum(len(qa['reasoning_steps']) for qa in dataset['qa_pairs']) / total_qa
    
    print(f"   总数量: {total_qa}")
    print(f"   平均问题长度: {avg_question_len:.0f} 字符")
    print(f"   平均答案长度: {avg_answer_len:.0f} 字符")
    print(f"   平均推理步骤: {avg_reasoning_steps:.1f} 步")
    
    # 文件分布
    source_files = [qa.get('source_file', 'unknown') for qa in dataset['qa_pairs']]
    file_dist = Counter(source_files)
    print(f"\n   文件覆盖数: {len(file_dist)}")
    print(f"   文件分布: {dict(list(file_dist.most_common(3)))}")
    
    # 示例展示
    print("\n   【示例问答对】")
    sample = dataset['qa_pairs'][0]
    print(f"   Q: {sample['question'][:80]}...")
    print(f"   A: {sample['answer'][:80]}...")
else:
    print("   ⚠️  未生成问答对")

# 统计设计方案
print("\n" + "="*70)
print("🏗️  设计方案质量分析")
print("="*70)

if dataset['design_solutions']:
    total_design = len(dataset['design_solutions'])
    avg_steps = sum(len(d['steps']) for d in dataset['design_solutions']) / total_design
    avg_files = sum(len(d['files_to_modify']) for d in dataset['design_solutions']) / total_design
    
    print(f"   总数量: {total_design}")
    print(f"   平均实施步骤: {avg_steps:.1f} 步")
    print(f"   平均修改文件: {avg_files:.1f} 个")
    
    # 示例展示
    print("\n   【示例设计方案】")
    sample = dataset['design_solutions'][0]
    print(f"   需求: {sample['requirement'][:60]}...")
    print(f"   方案: {sample['solution'][:60]}...")
else:
    print("   ⚠️  未生成设计方案")

print("\n" + "="*70)

📊 问答对质量分析
   总数量: 10
   平均问题长度: 497 字符
   平均答案长度: 4152 字符
   平均推理步骤: 12.3 步

   文件覆盖数: 5
   文件分布: {'generate_anatomy_mse_benchmark.py': 4, 'llm_matching.py': 2, 'fix_multifarm_reference.py': 2}

   【示例问答对】
   Q: The provided code snippet lists a series of paths such as `"conference/cmt-sigkd...
   A: The path structure exemplified by `"conference/cmt-sigkdd/component/"` holds sig...

🏗️  设计方案质量分析
   总数量: 5
   平均实施步骤: 7.6 步
   平均修改文件: 11.6 个

   【示例设计方案】
   需求: 扩展generate_conference_benchmark以支持弱网环境场景...
   方案: 为了扩展`generate_conference_benchmark`以支持弱网环境，核心策略将围绕**提高LLM AP...



## 💾 步骤7: 保存训练数据

将数据保存为JSON格式

In [12]:
from datetime import datetime

# 生成输出文件名
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
project_name = project_path.name
output_file = f"outputs/{project_name}/training_data_{timestamp}.json"

# 保存数据
generator.save_dataset(dataset, output_file)

# 验证文件
output_path = Path(output_file)
if output_path.exists():
    file_size = output_path.stat().st_size
    print(f"\n✅ 文件已保存")
    print(f"   路径: {output_path}")
    print(f"   大小: {file_size / 1024:.1f} KB")
    
    # 读取验证
    with open(output_path, 'r', encoding='utf-8') as f:
        loaded = json.load(f)
    print(f"   问答对: {len(loaded['qa_pairs'])}")
    print(f"   设计方案: {len(loaded['design_solutions'])}")


💾 数据已保存: outputs/ontology-llm/training_data_20251219_131646.json
   文件大小: 201.9 KB

✅ 文件已保存
   路径: outputs/ontology-llm/training_data_20251219_131646.json
   大小: 201.9 KB
   问答对: 10
   设计方案: 5


## 🎓 步骤8: 数据预览

查看完整的训练数据样本

In [16]:
import json

# 美化输出第一个问答对
if dataset['qa_pairs']:
    print("="*70)
    print("📝 完整问答对示例")
    print("="*70)
    sample_qa = dataset['qa_pairs'][0]
    print(json.dumps(sample_qa, ensure_ascii=False, indent=2))
    print("\n")

# 美化输出第一个设计方案
if dataset['design_solutions']:
    print("="*70)
    print("🏗️  完整设计方案示例")
    print("="*70)
    sample_design = dataset['design_solutions'][0]
    print(json.dumps(sample_design, ensure_ascii=False, indent=2))

📝 完整问答对示例
{
  "question": "The provided code snippet lists a series of paths such as `\"conference/cmt-sigkdd/component/\"` and `\"conference/confof-edas/component/\"`. Given the project `ontology-llm`, its purpose in working with ontologies, and core modules like `llm_om_zero_shot` and `run_series_conference`, what is the architectural significance of this path structure for organizing and executing ontology matching benchmarks, and how does `run_series_conference.py` leverage it to facilitate the systematic evaluation process?",
  "answer": "The path structure exemplified by `\"conference/cmt-sigkdd/component/\"` holds significant architectural importance within the `ontology-llm` project for organizing and automating ontology matching benchmarks.\n\n**Architectural Significance:**\n\n1.  **Modularization of Matching Tasks:** Each unique path (`conference/<ontology1>-<ontology2>/component/`) represents a distinct, self-contained ontology matching task. This structure effectively modu

## 🚀 快捷方式: 一键生成

如果不需要详细步骤，可以使用快捷函数

In [14]:
from src.simple_generator import quick_generate

# 一键生成（所有步骤自动完成）
dataset_quick = quick_generate(
    project_path=str(project_path),
    num_qa=5,
    num_design=3,
    output_path=f"outputs/{project_name}/quick_output.json",
    use_context=True
)

print("\n🎉 快捷生成完成！")

🚀 开始生成训练数据

📁 发现Python文件...
   找到 13 个文件

📝 生成 5 个问答对...
   [1/5] 处理文件: llm_om_few_shot.py
       ✅ 成功
   [2/5] 处理文件: om_csv_to_database.py
       ✅ 成功
   [3/5] 处理文件: om_ontology_to_csv.py
       ✅ 成功
   [4/5] 处理文件: fix_multifarm_reference.py
       ✅ 成功
   [5/5] 处理文件: generate_anatomy_mse_benchmark.py
       ✅ 成功

🏗️  生成 3 个设计方案...
   [1/3] 需求: 扩展om_csv_to_database以支持微服务场景...
       ✅ 成功
   [2/3] 需求: 扩展om_ontology_to_csv以支持边缘计算场景...
       ✅ 成功
   [3/3] 需求: 为generate_conference_benchmark添加批量处理功能...
       ✅ 成功

📊 生成完成
   问答对: 5/5
   设计方案: 3/3
   成功率: 100.0%

💾 数据已保存: outputs/ontology-llm/quick_output.json
   文件大小: 96.8 KB

🎉 快捷生成完成！


---

## 📚 附录: 进阶配置


### 常见问题

**Q: API调用失败怎么办？**
- 检查API密钥是否正确
- 验证网络连接
- 查看API配额限制

**Q: 如何提高生成质量？**
- 启用上下文增强 (`use_context=True`)
- 选择合适的上下文级别
- 调整temperature参数（0.2-0.4最佳）

**Q: 如何加速生成？**
- 使用`minimal`上下文级别
- 减少单次生成数量
- 使用更快的模型